In [1]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint


# todos
- [ ] get ships from contract
   * np.where + mod 
   * create a column `is_available` in `fleet` with fleet availability   
- [ ]create `contracts` df from `create_contracts()`.
   * Columns : `start_date`, `end_date`, `end_port`, `value_in_$`
   * `start_date` == start_step + (steps for pick up == dm / 14 knots)
   
   * `end_date` == `start_date` + (max_distance in dm / 10 knots) in steps
- [ ] create `can_reach` function that calculates if a ship can make it in time for the contract
   * maybe implement `can_reach` with 10,12,14 knots? 


## Agent
- [ ] Action Space:
   * Choose contract Discrete 4?
   * Choose ship Discrete 4?
   * Choose speed Discrete 3 [10,12,14]?
- [ ] 

In [9]:
# load data

ports = pd.read_csv('data/ports_10.csv')
dm = pd.read_csv('data/distance_matrix.csv')
fleet = pd.read_csv('data/fleet.csv')

In [5]:
fleet

,vessel_name,vessel_type,dwt,cii_required,cii_attained,current_port,current_speed,is_available
0,handy_1,handymax,40000,0,0,0,0,1
1,handy_2,handymax,40000,0,0,0,0,1
2,supra_1,supramax,50000,0,0,0,0,1
3,supra_2,supramax,50000,0,0,0,0,1
4,ultra_1,ultramax,60000,0,0,0,0,1
5,ultra_2,ultramax,60000,0,0,0,0,1
6,pana_1,panamax,70000,0,0,0,0,1
7,pana_2,panamax,70000,0,0,0,0,1
8,kamsar_1,kamsarmax,80000,0,0,0,0,1
9,kamsar_2,kamsarmax,80000,0,0,0,0,1


In [100]:
data_dict = {
    "fleet_path":'data/fleet.csv',
    "ports_path":'data/ports_10.csv',
    "dm_path" : 'data/distance_matrix.csv'
}

In [6]:
ports

,number,name,country,region,code,position,latitude,longitude,location
0,1,Shanghai,China,East Asia,CNSHA,"31° 15.500' N, 121° 32.600' E",31.258333,121.543333,Yangtze Delta
1,2,Singapore,Singapore,Southeast Asia,SGSIN,"1° 15.500' N, 103° 47.900' E",1.258330,103.798333,Singapore Strait
2,3,Xiamen,China,East Asia,CNXMN,"24° 27.100' N, 118° 4.300' E",24.451670,118.071667,Taiwan Strait
3,4,Dalian,China,East Asia,CNDLC,"38° 56.300' N, 121° 39.500' E",38.938330,121.658333,Yellow Sea
4,5,Guangzhou,China,East Asia,CNCAN,"23° 3.900' N, 113° 25.150' E",23.065000,113.419167,Pearl River Delta
5,6,Qingdao,China,East Asia,CNTAO,"36° 5.650' N, 120° 19.000' E",36.094170,120.316667,Yellow Sea
6,7,Busan,South Korea,East Asia,KRPUS,"35° 6.500' N, 129° 3.500' E",35.108330,129.058333,Korean Strait
7,8,Hong Kong,Hong Kong,East Asia,HKHKG,"22° 17.400' N, 114° 10.100' E",22.290000,114.168333,Pearl River Delta
8,9,Tianjin,China,East Asia,CNTXG,"38° 58.400' N, 117° 46.300' E",38.973330,117.771667,Yellow Sea
9,10,Kaohsiung,Taiwan,East Asia,TWKHH,"22° 34.800' N, 120° 18.000' E",22.580000,120.300000,Taiwan Strait


In [10]:
# set fleet at random ports

fleet['current_port'] =  np.random.randint(1,11,fleet.shape[0])
fleet

,vessel_name,vessel_type,dwt,cii_required,cii_attained,current_port,current_speed,is_available
0,handy_1,handymax,40000,0,0,4,0,1
1,handy_2,handymax,40000,0,0,2,0,1
2,supra_1,supramax,50000,0,0,10,0,1
3,supra_2,supramax,50000,0,0,5,0,1
4,ultra_1,ultramax,60000,0,0,8,0,1
5,ultra_2,ultramax,60000,0,0,9,0,1
6,pana_1,panamax,70000,0,0,3,0,1
7,pana_2,panamax,70000,0,0,6,0,1
8,kamsar_1,kamsarmax,80000,0,0,5,0,1
9,kamsar_2,kamsarmax,80000,0,0,4,0,1


In [11]:
# create random contracts
def create_contr(seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 50 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        contract_count = 4
        np.random.seed(seed)
        x = np.random.multinomial(contract_count, [1/5]*5, None)
        z = np.zeros(shape=45, dtype=np.int32)
        c = np.concatenate((x,z))
        np.random.shuffle(c)
        return c

In [85]:
y = create_contr(4)
print(f"y is {y}")
d = np.nonzero(y)[0]
print(f"d is {d}")
# if len(d) != 4:
#     if len(d) == 1:
#     (4)
#     # check for 4 contracts in one port
#     elif len(d) == 2: 
#     # check for 3 contracts in one port
#     (3,1),(2,2)(1,3)
#     # check for 2 contracts in one port
#     (1,1,2), (1,2,1), (2,1,1)

vals = y[d]
print(f"vals is {vals}")
fives = np.ones((len(d)))*5
print(fives)
x = np.mod(d,fives)
print(x)
conditions = [ x == 0 , x == 1, x == 2, x == 3, x == 4 ]
choices = ['handymax','supramax','ultramax','panamax','kamsarmax']
k = np.select(conditions,choices)
print(k)
print(len(k))
port_with_2 = np.where(y==2)
port_with_3 = np.where(y==3)
port_with_4 = np.where(y==4)
#if len(k) != 4: #check for double or triple contracts in a port 
if (len(k) == 2) and (2 in vals):
    print("Hi")
    k = np.array([k[0],k[0],k[1],k[1]])
    print(k)


y is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0
 0 0 2 0 0 0 0 0 0 0 0 0 0]
d is [35 39]
vals is [2 2]
[5. 5.]
[0. 4.]
['handymax' 'kamsarmax']
2
Hi
['handymax' 'handymax' 'kamsarmax' 'kamsarmax']


In [76]:
check_1 = np.where(y==1)
check_2 = np.where(y==2)
check_3 = np.where(y==3)
check_4 = np.where(y==4)


In [77]:
print(check_1)

(array([], dtype=int64),)


In [51]:
y = np.zeros(shape=(50,))
y[[0,2,4,5,6,7,11,15,22,28,32,36,44,47,49]] = len([0,2,4,5,6,7,11,15,22,28,32,36,44,47,49]) * [1] 
d = np.nonzero(y)[0]
d

array([ 0,  2,  4,  5,  6,  7, 11, 15, 22, 28, 32, 36, 44, 47, 49])

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [99]:
# ports_ship_contracts_df
p = ports['number']
rp = p.repeat(2)
print(rp)

f = fleet['vessel_type'].unique()
f_tiled = np.tile(f,2)
print(f_tiled)

0     1
0     1
1     2
1     2
2     3
2     3
3     4
3     4
4     5
4     5
5     6
5     6
6     7
6     7
7     8
7     8
8     9
8     9
9    10
9    10
Name: number, dtype: int64
['handymax' 'supramax' 'ultramax' 'panamax' 'kamsarmax' 'handymax'
 'supramax' 'ultramax' 'panamax' 'kamsarmax']


array(['handymax', 'ultramax', 'kamsarmax', 'handymax', 'supramax',
       'ultramax', 'supramax', 'handymax', 'ultramax', 'panamax',
       'ultramax', 'supramax', 'kamsarmax', 'ultramax', 'kamsarmax'],
      dtype='<U21')

In [20]:
ship_vec = np.zeros(shape=(50,),dtype=str)
ship_vec



array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
      dtype='<U1')

In [105]:
def get_ship_type(contrs):
    pass


In [108]:
import gym 
import pandas as pd
import numpy as np
from gym import spaces

class CarbonEnv(gym.Env):
    """
    A custom environment for the Carbon emission problem.
    Follows the openai gym logic 
    """
  

    def __init__(self,data_dict):
        super().__init__()
       


        self.fleet = pd.read_csv(data_dict['fleet_path'])
        self.ports = pd.read_csv(data_dict['ports_path'])
        self.dm = pd.read_csv(data_dict['dm_path'])
        # calculate fleet's required cii
        self.fleet['cii_required'] = self.fleet['dwt'].map(expected_cii)
        # set fleet at random ports
        self.fleet['current_port'] = np.random.randint(1,11,self.fleet.shape[0])



        self.observation_space = spaces.Dict({
            "contracts" : spaces.Box(low=0,high=4,shape=(50,), dtype=np.int32),
            "ships" : spaces.Discrete(10,),
            "speeds" : spaces.Box(low=0,high=2,shape=(10,), dtype=np.int32)
            })


        self.action_space = spaces.Dict({
            "choose_contract" : spaces.Discrete(4,), # choose 1,2,3 or 4 contracts
            "choose_ship" : spaces.Discrete(2,), # which of the 2 ships in the class to pick
            "choose_speed" : spaces.Discrete(3,) # which of the 3 speeds to pick
            #"do_nothing"

        })

    def set_obs_space(self):
        self.observation_space['contracts'] = self.create_contracts()
        

    def set_act_space(self,):
        pass


    def step(self, action):
        # Execute one time step in the environment
        pass

    def reset(self):
        # Reset the state of the environment to an initial state
        obs = self.create_contracts(seed=0)
        return obs
        


    def render(self, mode='human', close=False):
        # Render the environment to the screen
        pass

    
    def create_contracts(self, seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 50 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        contract_count = 4
        np.random.seed(seed)
        x = np.random.multinomial(contract_count, [1/5]*5, None)
        z = np.zeros(shape=45, dtype=np.int32)
        c = np.concatenate((x,z))
        np.random.shuffle(c)
        return c

    def expected_cii(dwt, year=2023):
        """
        a method to calculate the expected cii of a fleet given a year
        """
        a = 4745
        c = 0.622
        z = {2023: 5, 2024: 7, 2025: 8, 2026: 9}
        cii_ref = a * dwt ** (-c)
        cii_req = ((100-z[year]) / 100 ) * cii_ref
        return cii_req

    
 


In [109]:
envo = CarbonEnv(data_dict)

NameError: name 'cii_required' is not defined

In [37]:
envo.observation_space['contracts']

Box(0, 4, (50,), int32)

In [41]:
envo.reset()

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0])

In [ ]:
def can_reach(current_port, target_port, start_time, free):
    """
    function checking if a vessel can reach a start port on time
    """
    # if the vessel is free
    # the vessel has to reach the new start port on time
    # u = Dx / Dt 


    # # if the vessel is not free 
    # the vessel has to reach the previous end port on time # use set of speeds 

    # u = Dx / Dt 
    # and then
    # the vessel has to reach the new start port on time


    pass

In [ ]:
def can_serve():
    """
    function checking if a vessel meets the conditions to serve a contract 
    """
    # # Check if vessel has the capacity to transport the cargo
    # vessel_capacity >= cargo_volume
    
    #     # If it does then check if it is free or already serving a cargo
    #     vessel_free == Yes:

    #         # Check if the vessel can reach the start port at start_time
    #         can_reach()    





    pass